In [59]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer


from sklearn.linear_model import LogisticRegression
import re
import string

In [40]:
data_fake=pd.read_csv("Fake.csv")
data_true=pd.read_csv("True.csv")


In [41]:
data_fake["class"]=0
data_true["class"]=1

In [42]:
data_merge=pd.concat([data_fake,data_true],axis=0)
data_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [43]:
data_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [44]:
data=data_merge.drop(['title','subject','date'],axis=1)

In [45]:
data.isnull().sum()

,0
text,0
class,0


In [46]:
data=data.sample(frac=1)

In [47]:
data.head()

,text,class
10435,WASHINGTON (Reuters) - The Chicago Tribune end...,1
16971,BRUSSELS (Reuters) - European Union leaders ca...,1
6302,"Man, it has got to suck to be Mitt Romney. We ...",0
19530,WASHINGTON (Reuters) - The United States stro...,1
3673,RIYADH (Reuters) - President Donald Trump was ...,1


In [48]:
data.reset_index(inplace=True)
data.drop(['index'],axis=1,inplace=True)

In [49]:
data.columns
data.head()

,text,class
0,WASHINGTON (Reuters) - The Chicago Tribune end...,1
1,BRUSSELS (Reuters) - European Union leaders ca...,1
2,"Man, it has got to suck to be Mitt Romney. We ...",0
3,WASHINGTON (Reuters) - The United States stro...,1
4,RIYADH (Reuters) - President Donald Trump was ...,1


In [50]:
def wordopt(text):
  text= text.lower()
  text= re.sub('\[.*?\]', '', text)
  text= re.sub("\\W"," ", text)
  text= re.sub('https?://\S+|www\.\S+','',text)
  text= re.sub('[%s]' % re.escape(string.punctuation),'',text)
  text= re.sub('\n','', text)
  text= re.sub('\w*\d\w*','', text)
  return text

In [51]:
data['text']= data['text'].apply(wordopt)

In [52]:
x=data['text']
y=data['class']


In [66]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming x and y are already defined (x: features, y: labels)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

# Initialize the TfidfVectorizer
v = TfidfVectorizer()

# Fit and transform the training data
xv_train = v.fit_transform(x_train)

# Transform the test data (use transform here to avoid data leakage)
xv_test = v.transform(x_test)

# xv_train and xv_test now contain the transformed TF-IDF vectors


In [68]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [70]:
pred_lr= LR.predict(xv_test)


In [71]:
LR.score(xv_test,y_test)

0.9865478841870824

In [72]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      5808
           1       0.98      0.99      0.99      5417

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [74]:
from sklearn.tree import DecisionTreeClassifier
DT= DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [75]:
pred_dt=DT.predict(xv_test)

In [77]:
DT.score(xv_test, y_test)

0.9961692650334075

In [81]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5808
           1       1.00      1.00      1.00      5417

    accuracy                           1.00     11225
   macro avg       1.00      1.00      1.00     11225
weighted avg       1.00      1.00      1.00     11225



In [80]:
from sklearn.ensemble import GradientBoostingClassifier
GB= GradientBoostingClassifier(random_state=0)
GB.fit(xv_train,y_train)

GradientBoostingClassifier(random_state=0)

In [82]:
pred_gb=GB.predict(xv_test)

In [83]:
GB.score(xv_test,y_test)

0.99456570155902

In [84]:
print(classification_report(y_test, pred_gb))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      5808
           1       0.99      1.00      0.99      5417

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [87]:
from sklearn.ensemble import RandomForestClassifier
RF= RandomForestClassifier(random_state=0)
RF.fit(xv_train,y_train)

RandomForestClassifier(random_state=0)

In [88]:
pred_rf=RF.predict(xv_test)

In [89]:
RF.score(xv_test,y_test)

0.9885077951002227

In [90]:
print(classification_report(y_test, pred_rf))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5808
           1       0.99      0.99      0.99      5417

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [99]:
def output_lable(n):
  if n==0:
    return "Fake News"
  elif n==1:
    return "Not A Fake News"

def manual_testing(news):
  testing_news= {"text":[news]}
  new_def_test= pd.DataFrame(testing_news)
  new_def_test["test"]= new_def_test["text"].apply(wordopt)
  new_x_test= new_def_test["text"]
  new_xv_test= v.transform(new_x_test)
  pred_LR=LR.predict(new_xv_test)
  pred_DT=DT.predict(new_xv_test)
  pred_GB=GB.predict(new_xv_test)
  pred_RF=RF.predict(new_xv_test)
  return print("\n\nLR Prediction: {} \nDT Prediction: {} \nRF Prediction: {}".format(output_lable(pred_LR[0]),
                                                                                       output_lable(pred_GB[0]),
                                                                                       output_lable(pred_RF[0])))

In [102]:
news=str(input())
manual_testing(news)

Newspapers published every day are known as dailies, while those published once a week are known as weeklies. News and views carried in newspapers depend on their periodicity. Newspapers have a serialized number printed on their issues.  Newspapers are printed on very thin paper called newsprint. Newspapers generally have a number of pages and cover a large number of topics such as politics, society, economy, business, science, technology, environment and sports. There are also entertainment news items such as on theatre and films.  Now-a-days newspapers are also available and can be browsed on the internet. Newspapers are usually low priced publications so that they are accessible to the maximum number of people.  Newspapers carry authentic news. Reporters and correspondents work for newspaper offices. They report from the place where an event has taken place or the scene of action. They write their reports and file their stories that are edited by editors and then published in the pa